In [1]:
import tkinter as tk
import cv2,os
import csv
import numpy as np
from PIL import Image
import pandas as pd
import datetime
import time
from tkinter import messagebox
from tkinter import *
from email1 import mailing

In [2]:
def clear1():
    txt.delete(0, 'end')    
    res = ""
    message.configure(text= res)
    
def clear2():
    txt2.delete(0, 'end')
    res = ""
    message.configure(text= res)
    
def clear2():
    txt3.delete(0, 'end')
    res = ""
    message.configure(text= res)

In [3]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False

In [4]:
def TakeImages():        
    Id=(txt.get())
    name=(txt2.get())
    email=(txt3.get())
    if not Id:
        res="Please enter Id"
        message.configure(text = res)
        MsgBox = tk.messagebox.askquestion ("Warning","Please enter your ID properly",icon = 'warning')
        if MsgBox == 'no':
            tk.messagebox.showinfo('Message','Please go through the instructions properly')
    elif not name:
        res="Please enter Name"
        message.configure(text = res)
        MsgBox = tk.messagebox.askquestion ("Warning","Please enter your name properly",icon = 'warning')
        if MsgBox == 'no':
            tk.messagebox.showinfo('Message','Please go through the instructions properly')
        
    elif(is_number(Id) and name.isalpha()):
            cam = cv2.VideoCapture(0)
            harcascadePath = "haarcascade_frontalface_default.xml"
            detector=cv2.CascadeClassifier(harcascadePath)
            sampleNum=0
            while(True):
                ret, img = cam.read()
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = detector.detectMultiScale(gray, 1.3, 5)
                for (x,y,w,h) in faces:
                    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)        
                    sampleNum+=1
                    cv2.imwrite("TrainingImage\ "+name +"."+Id +'.'+ str(sampleNum) + ".jpg", gray[y:y+h,x:x+w])
                    cv2.imshow('frame',img)
                if cv2.waitKey(100) & 0xFF == ord('q'):
                    break
                elif sampleNum>90:
                    break
            cam.release()
            cv2.destroyAllWindows() 
            res = "Images Saved for ID : " + Id +" Name : "+ name
            message1=Id+' '+name+' '+email
            with open('contact.txt',mode='a+') as file:
                file.write('\n')
                file.write(message1)
            row = [Id , name, email]
            with open('EmployeeDetails\EmployeeDetails.csv',mode = 'a+') as csvFile:
                writer = csv.writer(csvFile)
                writer.writerow(row)
            csvFile.close()
            message.configure(text= res)
    else:
        if(is_number(Id)):
            res = "Enter Alphabetical Name"
            message.configure(text= res)
        if(name.isalpha()):
            res = "Enter Numeric Id"
            message.configure(text= res)

In [5]:
def TrainImages():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    faces,Id = getImagesAndLabels("TrainingImage")
    recognizer.train(faces, np.array(Id))
    recognizer.save("TrainingImageLabel\Trainner.yml")
    res = "Image Trained"
    clear1();
    clear2();
    message.configure(text= res)
    tk.messagebox.showinfo('Completed','Your model has been trained successfully!')

In [6]:
def getImagesAndLabels(path):

    imagePaths=[os.path.join(path,f) for f in os.listdir(path)] 
    
    faces=[]

    Ids=[]

    for imagePath in imagePaths:
        pilImage=Image.open(imagePath).convert('L')
        imageNp=np.array(pilImage,'uint8')
        Id=int(os.path.split(imagePath)[-1].split(".")[1])
        faces.append(imageNp)
        Ids.append(Id)        
    return faces,Ids

In [7]:
def TrackImages():
    recognizer = cv2.face.LBPHFaceRecognizer_create()#cv2.createLBPHFaceRecognizer()
    recognizer.read("TrainingImageLabel\Trainner.yml")
    harcascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath);    
    df=pd.read_csv("EmployeeDetails\EmployeeDetails.csv")
    cam = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX        
    col_names =  ['Id','Name','Date','Time']
    attendance = pd.DataFrame(columns = col_names)    
    while True:
        ret, im =cam.read()
        gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        faces=faceCascade.detectMultiScale(gray, 1.2,5)    
        for(x,y,w,h) in faces:
            cv2.rectangle(im,(x,y),(x+w,y+h),(225,0,0),2)
            Id, conf = recognizer.predict(gray[y:y+h,x:x+w])                                   
            if(conf < 50):
                ts = time.time()      
                date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                aa=df.loc[df['Id'] == Id]['Name'].values
                tt=str(Id)+"-"+aa
                attendance.loc[len(attendance)] = [Id,aa,date,timeStamp]
                
                #Automated E-Mail
                #mailing(Id)
                
            else:
                Id='Unknown'
                tt=str(Id)  
            if(conf > 75):
                noOfFile=len(os.listdir("ImagesUnknown"))+1
                cv2.imwrite("ImagesUnknown\Image"+str(noOfFile) + ".jpg", im[y:y+h,x:x+w])            
            cv2.putText(im,str(tt),(x,y+h), font, 1,(0,0,0),2)        
        attendance=attendance.drop_duplicates(subset=['Id'],keep='first')    
        cv2.imshow('image',im)
        if (cv2.waitKey(1)==ord('q')):
            break
    ts = time.time()      
    date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
    fileName="Attendance\Attendance_"+date+".csv"
    if fileName in os.listdir("Attendance/"):
        attendance.to_csv(fileName, mode='a', header=False,index= False)
    else:
        attendance.to_csv(fileName, index=False)
    cam.release()
    cv2.destroyAllWindows()
    res=attendance
    message2.configure(text= res)
    res = "Attendance Taken"
    message.configure(text= res)
    tk.messagebox.showinfo('Completed','Congratulations ! Your attendance has been marked successfully for the day!!')
    for i in list(attendance['Id']):
        mailing(str(i))

In [8]:
def quit_window():
   MsgBox = tk.messagebox.askquestion ('Exit Application','Are you sure you want to exit the application',icon = 'warning')
   if MsgBox == 'yes':
       tk.messagebox.showinfo("Greetings", "Thank You very much for using our software. Have a nice day ahead!!")
       window.destroy()

In [ ]:
window = tk.Tk()
from PIL import Image, ImageTk

window.title("Attendance System")

#window.configure(background='white')
window.geometry('1800x1200')

image = Image.open("bg.jpg")
image=image.resize((1800,1200), Image.ANTIALIAS)
photo = ImageTk.PhotoImage(image)
l2=tk.Label(image=photo)
l2.pack(ipadx="100",fill=BOTH)

window.grid_rowconfigure(0, weight=1)
window.grid_columnconfigure(0, weight=1)

checker=0;

message = tk.Label(window, text="Restart India Project" ,bg="white"  ,fg="black"  ,width=20  ,height=2,font=('Times New Roman', 25, 'bold')) 
message.place(x=400, y=76)

message = tk.Label(window, text="ATTENDANCE MANAGEMENT PORTAL" ,bg="white"  ,fg="black"  ,width=32  ,height=1,font=('Times New Roman', 35, 'bold underline')) 
message.place(x=110, y=20)

lbl = tk.Label(window, text="Employee ID",width=10  ,height=2  ,fg="black"  ,bg="white" ,font=('Times New Roman', 25, ' bold ') ) 
lbl.place(x=55, y=210)


txt = tk.Entry(window,width=20,bg="black" ,fg="white",font=('Times New Roman', 15, ' bold '))
txt.place(x=60, y=300)

lbl2 = tk.Label(window, text="Name",width=7  ,fg="black"  ,bg="white"    ,height=2 ,font=('Times New Roman', 25, ' bold ')) 
lbl2.place(x=378, y=210)

txt2 = tk.Entry(window,width=20  ,bg="black"  ,fg="white",font=('Times New Roman', 15, ' bold ')  )
txt2.place(x=360, y=300)

lbl4 = tk.Label(window, text="Email",width=7  ,fg="black"  ,bg="white"    ,height=2 ,font=('Times New Roman', 25, ' bold ')) 
lbl4.place(x=660, y=210)

txt3 = tk.Entry(window,width=20  ,bg="black"  ,fg="white",font=('Times New Roman', 15, ' bold ')  )
txt3.place(x=640, y=300)

lbl3 = tk.Label(window, text="NOTIFICATION",width=13  ,fg="black"  ,bg="white"  ,height=2 ,font=('Times New Roman', 25, ' bold ')) 
lbl3.place(x=890, y=220)

message = tk.Label(window, text="" ,bg="black"  ,fg="white"  ,width=20  ,height=1, activebackground = "white" ,font=('Times New Roman', 15, ' bold ')) 
message.place(x=900, y=300)

lbl3 = tk.Label(window, text="ATTENDANCE:",width=12  ,fg="black"  ,bg="white"  ,height=1 ,font=('Times New Roman', 30, ' bold ')) 
lbl3.place(x=200, y=540)


message2 = tk.Label(window, text="" ,fg="white"   ,bg="black",activeforeground = "green",width=60  ,height=4  ,font=('times', 15, ' bold ')) 
message2.place(x=520, y=520)

lbl4 = tk.Label(window, text="STEP 1",width=20  ,fg="black"  ,bg="white"  ,height=2 ,font=('Times New Roman', 20, ' bold '))
lbl4.place(x=110, y=375)

lbl5 = tk.Label(window, text="STEP 2",width=20  ,fg="black"  ,bg="white"  ,height=2 ,font=('Times New Roman', 20, ' bold ')) 
lbl5.place(x=515, y=375)

lbl6 = tk.Label(window, text="STEP 3",width=20  ,fg="black"  ,bg="white"  ,height=2 ,font=('Times New Roman', 20, ' bold ')) 
lbl6.place(x=930, y=362)
 


    
takeImg = tk.Button(window, text="IMAGE CAPTURE BUTTON", command=TakeImages  ,fg="black"  ,bg="white"  ,width=25  ,height=2, activebackground = "white" ,font=('Times New Roman', 15, ' bold '))
takeImg.place(x=110, y=425)
trainImg = tk.Button(window, text="MODEL TRAINING BUTTON", command=TrainImages  ,fg="black"  ,bg="white"  ,width=25  ,height=2, activebackground = "white" ,font=('Times New Roman', 15, ' bold '))
trainImg.place(x=520, y=425)
trackImg = tk.Button(window, text="ATTENDANCE MARKING BUTTON", command=TrackImages  ,fg="black"  ,bg="white"  ,width=30  ,height=3, activebackground = "white" ,font=('Times New Roman', 15, ' bold '))
trackImg.place(x=900, y=412)
quitWindow = tk.Button(window, text="QUIT", command=quit_window  ,fg="white"  ,bg="red"  ,width=10  ,height=1, activebackground = "black" ,font=('Times New Roman', 15, ' bold '))
quitWindow.place(x=1115, y=618)
 
    

window.mainloop()

Hello Mayank, Your attendance for today has been updated for ID: 9876. Have a wonderful day ahead.
